In [1]:
import pandas as pd

from prereise.gather.demanddata.nrel_efs.get_efs_data import (
    download_flexibility_data,
    partition_flexibility_by_sector,
)
from prereise.gather.demanddata.nrel_efs.map_states import (
    decompose_demand_profile_by_state_to_loadzone,
)

## Notebook Overview

In this notebook, the functionality of the various modules used for obtaining and cleaning the National Renewable Energy Laboratory's (NREL's) Electrification Futures Study (EFS) flexibility data is displayed. An example is developed using the **Reference** electrification scenario, **Slow** technology advancement, and **Base** flexibility scenario for the year **2030**. 

The NREL EFS flexibility data can be obtained from this [this website](https://data.nrel.gov/submissions/127), with the specific dataset required for the described example directly available [here](https://data.nrel.gov/system/files/127/EFS%20Flexible%20Load%20Profiles%20-%20Reference%20Electrification.zip). These datasets are generously provided by NREL, which is operated for the U.S. Department of Energy by the Alliance for Sustainable Energy, LLC. Before using these datasets, please read [this disclaimer](https://www.nrel.gov/disclaimer.html) first.

## Downloading and Extracting EFS Flexibility Data

NREL EFS flexibility data is accessed from the website referenced in the prior section. The website contains a .zip file for each of the three electrification scenarios (Reference, Medium, and High). Each .zip file contains a .csv file containing the sectoral demand data for each state in the contiguous U.S., each level of technology advancement (Slow, Moderate, and Rapid), each flexibility scenario (Base and Enhanced), and each of the six tested years (2018, 2020, 2024, 2030, 2040, and 2050). The `download_flexibility_data` function downloads the .zip file of NREL EFS data and attempts to extract the .csv file. `download_flexibility_data` can download NREL EFS data for one electrification scenario:

In [2]:
download_flexibility_data(es={"Reference"}, fpath="")

EFS Flexible Load Profiles - Reference Electrification.zip successfully downloaded!
EFS Flexible Load Profiles - Reference Electrification.zip is compressed using a method that is not supported by the zipfile module.
Trying other extraction methods supported by your OS.
EFSFlexLoadProfiles_Reference.csv successfully extracted!


`download_flexibility_data` can download EFS data for multiple electrification scenarios:

In [3]:
download_flexibility_data(es={"Medium", "High"}, fpath="")

EFS Flexible Load Profiles - Medium Electrification.zip successfully downloaded!
EFS Flexible Load Profiles - High Electrification.zip successfully downloaded!
EFS Flexible Load Profiles - Medium Electrification.zip is compressed using a method that is not supported by the zipfile module.
Trying other extraction methods supported by your OS.
EFSFlexLoadProfiles_Medium.csv successfully extracted!
EFSFlexLoadProfiles_High.csv successfully extracted!


`download_flexibility_data` can also download EFS data for all of the electrification scenarios:

In [4]:
download_flexibility_data(es={"All"}, fpath="")

EFS Flexible Load Profiles - High Electrification.zip successfully downloaded!
EFS Flexible Load Profiles - Medium Electrification.zip successfully downloaded!
EFS Flexible Load Profiles - Reference Electrification.zip successfully downloaded!
EFS Flexible Load Profiles - High Electrification.zip is compressed using a method that is not supported by the zipfile module.
Trying other extraction methods supported by your OS.
EFSFlexLoadProfiles_High.csv successfully extracted!
EFSFlexLoadProfiles_Medium.csv successfully extracted!
EFSFlexLoadProfiles_Reference.csv successfully extracted!


The .zip files were created with a compression format (Deflate64) that is not supported by Python's `zipfile` module. Therefore, to perform automated extraction of the .csv files, Command Line- or Terminal-level tools must be used. For users running macOS or Linux, the Terminal can extract the .csv file. Unfortunately for Windows users, the Command Line extraction tools do not support Deflate64. [7-Zip](https://www.7-zip.org/), the popular file archiver tool, can extract the .csv file; by specifying the file path of 7-Zip when calling `download_flexibility_data`, Windows users can also use the automated .csv file extraction. If the automated extraction techniques do not work (e.g., a Windows user does not have 7-Zip), the .zip file is still downloaded and can be extracted manually (e.g., using the extraction tool built into Windows' File Explorer).

## Splitting the EFS Flexibility by Sector, Technology Advancement, Flexibility Scenario, and Year

The EFS flexibility data for a given electrification scenario is provided for each sector, year, technology advancement, and flexibility scenario. However, it is useful to split flexibility data by sector because different sectors may have different operational constraints (e.g., duration between demand curtailment and recovery, directionality of load shift). The `partition_flexibility_by_sector` function filters EFS flexibility data for a specified year, technology advancement, and flexibility scenario and separates the state-level demand for each sector. `partition_flexibility_by_sector` can either access the extracted .csv file or can call `download_flexibility_data` to obtain the flexibility data; the extracted .csv file is searched for according to the provided electrification scenario (`es`) in the file path (`fpath`) provided by the user:

In [5]:
sect_flex = partition_flexibility_by_sector(
    es="Reference", ta="Slow", flex="Base", year=2030, fpath=""
)

print(sect_flex.keys())
print(sect_flex["Transportation"])

EFS Flexible Load Profiles - Reference Electrification.zip successfully downloaded!
EFS Flexible Load Profiles - Reference Electrification.zip is compressed using a method that is not supported by the zipfile module.
Trying other extraction methods supported by your OS.
EFSFlexLoadProfiles_Reference.csv successfully extracted!
dict_keys(['Commercial', 'Residential', 'Industrial', 'Transportation'])
State                       AL        AR         AZ         CA         CO  \
Local Time                                                                  
2016-01-01 00:00:00   4.155655  2.478171   6.805127  19.382927   4.113119   
2016-01-01 01:00:00   2.401389  1.006649   5.513717   8.074698   1.656038   
2016-01-01 02:00:00   1.834609  0.830685   2.580403   6.456153   1.409955   
2016-01-01 03:00:00   0.824335  0.586025   2.059998   4.238941   0.997348   
2016-01-01 04:00:00   0.719887  0.134069   1.226499   1.251424   0.211859   
...                        ...       ...        ...        

`partition_flexibility_by_sector` can also retain only a subset of sectoral flexibility data, if desired:

In [6]:
sect_flex = partition_flexibility_by_sector(
    es="Reference", 
    ta="Slow",
    flex="Base",
    year=2030, 
    sect={"Commercial", "Industrial"}, 
    fpath="",
)

print(sect_flex.keys())

EFS Flexible Load Profiles - Reference Electrification.zip successfully downloaded!
EFS Flexible Load Profiles - Reference Electrification.zip is compressed using a method that is not supported by the zipfile module.
Trying other extraction methods supported by your OS.
EFSFlexLoadProfiles_Reference.csv successfully extracted!
dict_keys(['Commercial', 'Industrial'])


Note that `partition_flexibility_by_sector` calls the `account_for_leap_year` function to create an extra day's worth of data (the EFS data sets have 8760-hour profiles whereas Breakthrough Energy uses 8784-hour profiles). To account for the extra day, flexibility data from January 2nd is copied and added to the end of the data set to create flexibility data for December 31st; January 2nd is chosen since it occurs on the same day of the week as December 31st (in a leap year), so as to preserve any weekly trends that may be present.

`partition_flexibility_by_sector` has the ability to save the resulting sectoral flexibility DataFrames, though it is not enabled by default. To save the DataFrames, set `save=True` when calling `partition_flexibility_by_sector`.

## Mapping the State Flexibility to the Appropriate Load Zones

Breakthrough Energy's production cost model requires flexibility to be specified for each load zone. The production cost model also considers operation in terms of UTC time, meaning that the flexibility provided according to each states' local time must be converted. The `decompose_demand_profile_by_state_to_loadzone` function takes a DataFrame of 8784-hour state-level flexibility and produces a DataFrame of 8784-hour load zone-level flexibility. `decompose_demand_profile_by_state_to_loadzone` calls the `shift_local_time_by_loadzone_to_utc` function, which is responsible for shifting the flexibility data (provided with respect to each state's local time) to be in terms of UTC time. Since this function causes the flexibility to shift, the last few hours' worth of flexibility are discarded and the first few hours' worth of flexibility are empty. To account for the first few hours' worth of flexibility (i.e., first five hours for EST load zones and first eight hours for PST load zones), the corresponding flexibility from the morning of December 30th is copied and added to the empty flexibility slots. Similar to the reasoning used in `account_for_leap_year`, December 30th was chosen since it occurs on the same day of the week as January 1st (in a leap year).

In [7]:
sect_flex = partition_flexibility_by_sector(
    es="Reference", ta="Slow", flex="Base", year=2030, fpath=""
)
sect_flex_lz = {
    k: decompose_demand_profile_by_state_to_loadzone(
        df=v, profile_type="demand_flexibility"
    )
    for k, v in sect_flex.items()
}

print(sect_flex_lz["Transportation"])

EFS Flexible Load Profiles - Reference Electrification.zip successfully downloaded!
EFS Flexible Load Profiles - Reference Electrification.zip is compressed using a method that is not supported by the zipfile module.
Trying other extraction methods supported by your OS.
EFSFlexLoadProfiles_Reference.csv successfully extracted!
Reading bus.csv
Reading plant.csv
Reading gencost.csv
Reading branch.csv
Reading dcline.csv
Reading sub.csv
Reading bus2sub.csv
Reading zone.csv
                       zone.1    zone.2    zone.3     zone.4    zone.5  \
UTC Time                                                                 
2016-01-01 00:00:00  4.491140  4.233990  2.350403  20.020450  2.359207   
2016-01-01 01:00:00  3.773710  3.561734  1.977204  17.036360  2.012111   
2016-01-01 02:00:00  3.104323  2.942024  1.641418  14.150630  1.656902   
2016-01-01 03:00:00  2.472214  2.345708  1.337440  11.257375  1.307822   
2016-01-01 04:00:00  1.673279  1.554680  0.893560   7.581477  0.869688   
...     

Note that the above is the most direct path to acquiring EFS flexibility data for a particular electrification scenario, technology advancement, flexibility scenario, and year and formatting it for use in the Breakthrough Energy production cost model. Unlike the base demand case (which is described in a separate demo notebook), flexibility data for each state and time step is not aggregated, but instead kept separate for each sector. As mentioned above, this is useful since different sectors may have different operational characteristics. `decompose_demand_profile_by_state_to_loadzone` can also be used to create a profile for a user-specified combination of interconnections and states.



In [8]:
sect_flex = partition_flexibility_by_sector(
    es="Reference", ta="Slow", flex="Base", year=2030, fpath=""
)
sect_flex_lz = {
    k: decompose_demand_profile_by_state_to_loadzone(
        df=v, profile_type="demand_flexibility", regions=["Texas"]
    )
    for k, v in sect_flex.items()
}

print(sect_flex_lz["Transportation"])

EFS Flexible Load Profiles - Reference Electrification.zip successfully downloaded!
EFS Flexible Load Profiles - Reference Electrification.zip is compressed using a method that is not supported by the zipfile module.
Trying other extraction methods supported by your OS.
EFSFlexLoadProfiles_Reference.csv successfully extracted!
                     zone.301  zone.302  zone.303  zone.304   zone.305  \
UTC Time                                                                 
2016-01-01 00:00:00  1.698612  1.915501  2.178095  8.776090  28.938050   
2016-01-01 01:00:00  1.725812  1.946174  2.212973  8.916621  29.401434   
2016-01-01 02:00:00  1.483687  1.673133  1.902501  7.665651  25.276518   
2016-01-01 03:00:00  1.240065  1.398404  1.590110  6.406949  21.126106   
2016-01-01 04:00:00  0.950667  1.072055  1.219021  4.911741  16.195846   
...                       ...       ...       ...       ...        ...   
2016-12-31 19:00:00  0.778028  0.877371  0.997649  4.019776  13.254706   
2016-

`decompose_demand_profile_by_state_to_loadzone` has the ability to save the resulting flexibility DataFrame, though it is not enabled by default. To save the DataFrame, set `save` equal to a valid file path and file name when calling `decompose_demand_profile_by_state_to_loadzone`.